## lenstronomy lens equation solver and flux ratios computation example notebook

In [1]:
# import standard python modules
import copy
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid, make_axes_locatable
%matplotlib inline


import lenstronomy
# import the lens model class 
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions
# import the lens equation solver class (finding image plane positions of a source position)
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
# import lens model solver with 4 image positions constrains
from lenstronomy.LensModel.Solver.solver4point import Solver4Point #Solver4Point
from lenstronomy.LensModel.Solver.solver2point import Solver2Point #Solver4Point

/Users/hskobe/anaconda3/lib/python3.11/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [2]:
# chose a lens model (list of parameterized lens models)
lens_model_list = ['EPL', 'SHEAR_GAMMA_PSI'] #['EPL'] #, 'SHEAR_GAMMA_PSI'] #, 'NFW', 'NFW']

#initialize classes with the specific lens models
lensModel = LensModel(lens_model_list)
lensEquationSolver = LensEquationSolver(lensModel)
solver2Point = Solver2Point(lensModel=lensModel, solver_type='THETA_E_ELLIPSE')
solver4Point = Solver4Point(lensModel=lensModel, solver_type='PROFILE_SHEAR')

### DCLS1507+0522

In [3]:
# # DCLS1507+0522: source 1, F200LP

x_image_s1_F200LP = np.array([-0.8344498087021135, 1.2285433511852109, -1.3986311119335793, -3.6291087887239137])
y_image_s1_F200LP = np.array([-1.9958577504452544, 4.1607764484457945, 4.312169573523052, 3.4710665194784394])

# x_image_s1_F200LP = np.array([1.2285433511852109, -1.3986311119335793, -3.6291087887239137])
# y_image_s1_F200LP = np.array([4.1607764484457945, 4.312169573523052, 3.4710665194784394])


kwargs_spemd = {'theta_E': 3.3544438046555687,
                'gamma': 2.3,
                'e1':  -0.49201384298672024,
                'e2': -0.06120134582696722,
                # 's_scale': 0.2,
                'center_x': -0.8166683329795008,
                'center_y': -0.5153087665158571}
kwargs_shear = {'gamma_ext': 0.24231283702646892,
                'psi_ext': 0*3.9746334374006644,
                'ra_0': 0,
                'dec_0': 0}

kwargs_lens_list = [kwargs_spemd, kwargs_shear]

##########################################################################################################
##########################################################################################################

kwargs_fit_s1, precision_s1 = solver4Point.constraint_lensmodel(x_pos=x_image_s1_F200LP, y_pos=y_image_s1_F200LP, 
                                                          kwargs_list=kwargs_lens_list, xtol=1.49012e-12)

print('kwargs_fit_s1:', kwargs_fit_s1)
print('#'*15)
print('#'*15)

##########################################################################################################
##########################################################################################################

for x, y in zip(x_image_s1_F200LP, y_image_s1_F200LP):
    
    x_image, y_image = x, y
    print('initial image position:', x_image, y_image)

    alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_fit_s1)

    x_source, y_source = x_image-alpha_x, y_image-alpha_y

    print('source:', x_source, y_source)

    x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_fit_s1, sourcePos_x=x_source, sourcePos_y=y_source, 
                                                                    min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
    print('image positions: ', x_image, y_image)
    print('#'*15)

kwargs_fit_s1: [{'theta_E': 3.649086989505014, 'gamma': 2.3, 'e1': 0.03437436103847938, 'e2': 0.1592923207942695, 'center_x': -0.9696202081433359, 'center_y': 0.28317996711959315}, {'gamma_ext': 0.2423128370264689, 'psi_ext': 0.2591076761404669, 'ra_0': 0, 'dec_0': 0}]
###############
###############
initial image position: -0.8344498087021135 -1.9958577504452544
source: -0.9888715795497337 1.875489644428035
image positions:  [ 1.22854335 -3.62910879 -1.39863111 -0.83444981] [ 4.16077645  3.47106652  4.31216957 -1.99585775]
###############
initial image position: 1.2285433511852109 4.1607764484457945
source: -0.9888715795497314 1.8754896444280362
image positions:  [ 1.22854335 -3.62910879 -1.39863111 -0.83444981] [ 4.16077645  3.47106652  4.31216957 -1.99585775]
###############
initial image position: -1.3986311119335793 4.312169573523052
source: -0.9888715795497326 1.8754896444280362
image positions:  [ 1.22854335 -3.62910879 -1.39863111 -0.83444981] [ 4.16077645  3.47106652  4.312169

In [13]:
kwargs_fit_s1

[{'theta_E': 3.649086989505014,
  'gamma': 2.3,
  'e1': 0.03437436103847938,
  'e2': 0.1592923207942695,
  'center_x': -0.9696202081433359,
  'center_y': 0.28317996711959315},
 {'gamma_ext': 0.2423128370264689,
  'psi_ext': 0.2591076761404669,
  'ra_0': 0,
  'dec_0': 0}]

In [20]:
lens_model_list_2 = ['EPL', 'SHEAR_GAMMA_PSI', 'SIS'] #, 'EPL']

lensModel2 = LensModel(lens_model_list_2)
lensEquationSolver2 = LensEquationSolver(lensModel2)
solver2Point2 = Solver2Point(lensModel=lensModel2, solver_type='THETA_E_ELLIPSE')
solver4Point2 = Solver4Point(lensModel=lensModel2, solver_type='PROFILE_SHEAR')

kwargs_lens2 = [
                  {'theta_E': 4.1067482538082185, 
                  'gamma': 2.189596543757959, 
                  'e1': -0.3129019644349088, 
                  'e2': 0.010355937418648315, 
                  'center_x': -0.8324189467458393, 
                  'center_y': -0.4891736887270096}, 
                  {'gamma_ext': 0.15655452321287866, 
                  'psi_ext': 0.35437003844812476, 
                  'ra_0': 0, 
                  'dec_0': 0}, 
                  {'theta_E': 0.011237730251575382, 
                    'center_x': -0.4213834534641387, 
                    'center_y': 1.1312497885360722}, 
                  # {'theta_E': 3.8667055630638005, 
                  #   'gamma': 2.093471094434474, 
                  #   'e1': -0.09096364243839426, 
                  #   'e2': 0.17224586380444912, 
                  #   'center_x': -0.9990742939346868, 
                  #   'center_y': 1.8750116433648205}
                  ] 

In [21]:
# DCLS1507+0522: source 2, F200LP

x_image_s2_F200LP = np.array([4.6994940741468145, -2.0628497614326937, -3.525668471279435, -2.298570140471534])
y_image_s2_F200LP = np.array([0.455225425191621, -2.489343251353538, -0.9910339637415673, 1.0811589757279219])

kwargs_spemd_s2 = {'theta_E': 3.3544438046555687,
                'gamma': 2.3,
                'e1':  -0.49201384298672024,
                'e2': -0.06120134582696722,
                's_scale': 0.2,
                'center_x': -0.8166683329795008,
                'center_y': -0.5153087665158571}
kwargs_shear_s2 = {'gamma_ext': 0.24231283702646892,
                'psi_ext': 0*3.9746334374006644,
                'ra_0': 0,
                'dec_0': 0}


kwargs_lens_list_s2 = [kwargs_spemd_s2, kwargs_shear_s2]

##########################################################################################################
##########################################################################################################

kwargs_fit_s2, precision_s2 = solver4Point2.constraint_lensmodel(x_pos=x_image_s2_F200LP, y_pos=y_image_s2_F200LP, 
                                                          kwargs_list=kwargs_lens2, xtol=1.49012e-12)

print('kwargs_fit_s2:', kwargs_fit_s2)
print('#'*15)
print('#'*15)

##########################################################################################################
##########################################################################################################

for x, y in zip(x_image_s2_F200LP, y_image_s2_F200LP):
    x_image, y_image = x, y
    print('initial image position:', x_image, y_image)

    alpha_x, alpha_y = lensModel2.alpha(x_image, y_image, kwargs_fit_s2)

    x_source, y_source = x_image-alpha_x, y_image-alpha_y

    print('source:', x_source, y_source)

    x_image, y_image = lensEquationSolver2.image_position_from_source(kwargs_lens=kwargs_fit_s2, sourcePos_x=x_source, sourcePos_y=y_source, 
                                                                    min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
    print('image positions: ', x_image, y_image)
    print('#'*15)


kwargs_fit_s2: [{'theta_E': 8.784979259161105, 'gamma': 2.189596543757959, 'e1': -0.9882432001078333, 'e2': 0.0801244532517847, 'center_x': -1.897247197621578, 'center_y': -0.7256726624824649}, {'gamma_ext': 0.15655452321287866, 'psi_ext': 0.8704397855672518, 'ra_0': 0, 'dec_0': 0}, {'theta_E': 0.011237730251575382, 'center_x': -0.4213834534641387, 'center_y': 1.1312497885360722}]
###############
###############
initial image position: 4.6994940741468145 0.455225425191621
source: 1.1849785554213996 -0.34717688469290553
image positions:  [ 4.69949407 -3.52566847 -2.29857014 -2.06284976] [ 0.45522543 -0.99103396  1.08115898 -2.48934325]
###############
initial image position: -2.0628497614326937 -2.489343251353538
source: 1.1849785554213454 -0.34717688469292973
image positions:  [ 4.69949407 -3.52566847 -2.29857014 -2.06284976] [ 0.45522543 -0.99103396  1.08115898 -2.48934325]
###############
initial image position: -3.525668471279435 -0.9910339637415673
source: 1.184978555421298 -0.3471

In [22]:
kwargs_fit_s2

[{'theta_E': 8.784979259161105,
  'gamma': 2.189596543757959,
  'e1': -0.9882432001078333,
  'e2': 0.0801244532517847,
  'center_x': -1.897247197621578,
  'center_y': -0.7256726624824649},
 {'gamma_ext': 0.15655452321287866,
  'psi_ext': 0.8704397855672518,
  'ra_0': 0,
  'dec_0': 0},
 {'theta_E': 0.011237730251575382,
  'center_x': -0.4213834534641387,
  'center_y': 1.1312497885360722}]

In [23]:
lens_model_list = ['EPL', 'SHEAR_GAMMA_PSI'] 

lensModel = LensModel(lens_model_list)
lensEquationSolver = LensEquationSolver(lensModel)

x_image, y_image = kwargs_source[4]['center_x'], kwargs_source[4]['center_y']

# alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_lens)

# x_source, y_source = x_image-alpha_x, y_image-alpha_y

# compute image position of full model (with numerical options attached)
x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens, 
                                                                    sourcePos_x=x_image, # x_source, 
                                                                    sourcePos_y=y_image, # y_source, 
                                                                    min_distance=0.01, 
                                                                    search_window=5, 
                                                                    precision_limit=10**(-10), 
                                                                    num_iter_max=100)

NameError: name 'kwargs_source' is not defined

In [ ]:
# from itertools import combinations

# def source_pos_dist(var, x_pos, y_pos):

#     x_source = []
#     y_source = []
#     dist = []

#     # x_pos = img_pos[:4]
#     # y_pos = img_pos[4:]

#     theta_e, gamma_ext, psi_ext = var
    
#     kwargs_spemd = {'theta_E': theta_e, 
#                     'gamma': 2.3093837411186064, 
#                     'e1': -0.14847497179603944, 
#                     'e2': -0.057853144396770605, 
#                     's_scale': 0.49948142237656606,
#                     'center_x': -0.8018140749005013, 
#                     'center_y': -0.49154401777238305}
#     kwargs_shear = {'gamma_ext': gamma_ext, 
#                     'psi_ext': psi_ext,
#                     'ra_0': 0,
#                     'dec_0': 0}

#     kwargs_list = [kwargs_spemd, kwargs_shear]

#     for x, y in zip(x_pos, y_pos):

#         alpha_x, alpha_y = lensModel.alpha(x, y, kwargs_list)

#         x_source.append(x - alpha_x)

#         y_source.append(y - alpha_y)

#     for i, j in combinations(zip(x_source, y_source), 2):
#         dist.append(np.sqrt( (j[0] - i[0])**2 + (j[1] - i[1])**2 ))

#     return np.sum(dist)

In [ ]:
# from scipy.optimize import minimize

# var = 3.7417482816708825, 0.21092436968446868, 0.07242467909933327

# res = minimize(source_pos_dist, var, args=(x_image_s2_F200LP, y_image_s2_F200LP), method='nelder-mead', options={'disp': True})

In [ ]:
# res.x

In [ ]:
# x = np.array([2.84880786, -2.18611749, -3.84199755, -2.61547983])
# y = np.array([0.44773636, -2.41528143, -0.99530428,  1.24961864])

In [ ]:
# kwargs_spemd = {'theta_E': 3.58878847, 
#                     'gamma': 2.3093837411186064, 
#                     'e1': -0.14847497179603944, 
#                     'e2': -0.057853144396770605, 
#                     's_scale': 0.49948142237656606,
#                     'center_x': -0.8018140749005013, 
#                     'center_y': -0.49154401777238305}
# kwargs_shear = {'gamma_ext': 0.21382391, 
#                 'psi_ext': 0.21382391,
#                 'ra_0': 0,
#                 'dec_0': 0}

# kwargs_list = [kwargs_spemd, kwargs_shear]

In [ ]:
# kwargs_fit, precision = solver4Point.constraint_lensmodel(x_pos=x, y_pos=y, 
#                                                           kwargs_list=kwargs_list, xtol=1.49012e-12)

# kwargs_fit

### DCLS0854-0424

In [ ]:
# # DCLS0854-0424

# # x_source_1 = -0.5646987053268822 - (2*0.01)
# # y_source_1 = 0.08835888499899273 + (22*0.01)

# # x_source_2 = -0.343 - (53*0.01)
# # y_source_2 = 0.08835888499899273 + (7*0.01)

# # x_source_3 = -0.343 - (87*0.01)
# # y_source_3 = 0.0393 + (25*0.01)

# x_source_4 = -0.5646987053268822
# y_source_4 = 0.08835888499899273

# # print(x_source_1, y_source_1)
# # print(x_source_2, y_source_2)
# # print(x_source_3, y_source_3)

# kwargs_pemd = {'theta_E': 2.404166080467353,
#                'gamma': 1.8233280499535798,
#                'e1': -0.059740982360995704,
#                'e2': 0.1342377290890493,
#                'center_x': 0.014621001392852512,
#                'center_y': 0.08923949467809739}
# kwargs_shear = {'gamma_ext': 0.10897252162554853,
#                 'psi_ext': -0.81856514273551,
#                 'ra_0': 0,
#                 'dec_0': 0}

# kwargs_lens_list = [kwargs_pemd, kwargs_shear]

# # x_image_1, y_image_1 = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source_1, sourcePos_y=y_source_1, 
# #                                                                  min_distance=0.01, search_window=5, precision_limit=10**(-10), num_iter_max=100)
# # print('image 1 positions: ', x_image_1, y_image_1)

# # x_image_2, y_image_2 = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source_2, sourcePos_y=y_source_2, 
# #                                                                  min_distance=0.01, search_window=5, precision_limit=10**(-10), num_iter_max=100)
# # print('image 2 positions: ', x_image_2, y_image_2)

# # x_image_3, y_image_3 = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source_3, sourcePos_y=y_source_3, 
# #                                                                  min_distance=0.01, search_window=5, precision_limit=10**(-10), num_iter_max=100)
# # print('image 3 positions: ', x_image_3, y_image_3)

# x_image_4, y_image_4 = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source_4, sourcePos_y=y_source_4, 
#                                                                  min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
# print('image 4 positions: ', x_image_4, y_image_4)

In [ ]:
# x_image, y_image = -2.754930458103047, -0.5003156080121595

# alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_lens_list)

# x_source, y_source = x_image-alpha_x, y_image-alpha_y

# print(x_source, y_source)

# x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, sourcePos_x=x_source, sourcePos_y=y_source, 
#                                                                  min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
# print('image positions: ', x_image, y_image)

F140W Info:

Lens center: 0.6491 0.1451
Image 1, 2 RA: 2.1056694526151 -2.1446663684802947
Image 1, 2 DEC: 0.16656821131187893 -0.09666993968892257
theta E estimate: 2.129037341147402

In [ ]:
# x_image, y_image known values
# x_image_F140W = [2.1056694526151, -2.1446663684802947]
# y_image_F140W = [0.16656821131187893, -0.09666993968892257]

# print('F140W image positions: ', x_image_F140W)

F200LP Info:

Lens center: 0.6455 0.1236
Image 1, 2 RA: 2.097939804907363 -2.1223084092251288
Image 1, 2 DEC: 0.13923809099998685 -0.09555221154370663
theta E estimate: 2.1132025459004162

In [ ]:
# x_image, y_image known values
# x_image_F200LP = [2.097939804907363, -2.1223084092251288]
# y_image_F200LP = [0.13923809099998685, -0.09555221154370663]

# print('F200LP image positions: ', x_image_F200LP)

In [ ]:
# fit the same lens model parameterization to the 4 image positions (free SPEMD model)
# the initial guess of the model can be rather different but 'gamma' has to be kept fixed
# kwargs_lens_init = copy.deepcopy(kwargs_lens_list)
# 'theta_E': 2.311432821867558, 'e1': -0.18785019961075974, 'e2': -0.12607809088893723
# kwargs_lens_init_F140W = [{'theta_E': 2.129037341147402, 'gamma': 2., 'e1': -0.1581155373378608, 'e2': -0.10765765074677205, 'center_x': 0.6491, 'center_y': 0.1451}]

# kwargs_fit_F140W, precision_F140W = solver2Point.constraint_lensmodel(x_pos=x_image_F140W, y_pos=y_image_F140W, 
#                                                           kwargs_list=kwargs_lens_init_F140W, xtol=1.49012e-10)

# kwargs_lens_init_F200LP = [{'theta_E': 2.1132025459004162, 'gamma': 2., 'e1': -0.19999967644529903, 'e2':  0.0258232185418328, 'center_x': 0.6455, 'center_y': 0.1236}]

# kwargs_fit_F200LP, precision_F200LP = solver2Point.constraint_lensmodel(x_pos=x_image_F200LP, y_pos=y_image_F200LP, 
#                                                           kwargs_list=kwargs_lens_init_F200LP, xtol=1.49012e-10)

In [ ]:
# x_source_new_F140W, y_source_new_F140W = lensModel.ray_shooting(x_image_F140W, y_image_F140W, kwargs_fit_F140W)
# print('F140W source center:', x_source_new_F140W, y_source_new_F140W)

# x_source_new_F200LP, y_source_new_F200LP = lensModel.ray_shooting(x_image_F200LP, y_image_F200LP, kwargs_fit_F200LP)
# print('F200LP source center:', x_source_new_F200LP, y_source_new_F200LP)

In [ ]:
# kwargs_fit_F140W

In [ ]:
# kwargs_fit_F200LP

### DESI-118.8480+34.7610

In [ ]:
# # # DESI-118.8480+34.7610: F606W

# # Image 1: (1.3548411332053694, 1.2684487670194255)
# # Image 2: (-0.7992269031586487, 2.2488218940396862)
# # Image 3: (-1.907215173224024, 0.16167868755750048)
# # Image 4: (1.3939117056447436, -1.7841665845670103)

# x_image_s1_F200LP = np.array([1.3548411332053694, -0.7992269031586487, -1.907215173224024, 1.3939117056447436])
# y_image_s1_F200LP = np.array([1.2684487670194255, 2.2488218940396862, 0.16167868755750048, -1.7841665845670103])

# # # DCLS1507+0522_local_customlogL_fitF200LP_s1_run_11
# kwargs_pemd = {'theta_E': 2.217815519520389,
#                 'gamma': 2.229256699205595,
#                 'e1': 0.19883947353154727,
#                 'e2': -0.10222795879338925,
#                 'center_x': -0.007,
#                 'center_y': 0.4194}
# kwargs_shear = {'gamma_ext': 0.020453977173297545,
#                 'psi_ext': 1.1857456084047209,
#                 'ra_0': 0,
#                 'dec_0': 0}

# kwargs_lens_list = [kwargs_pemd, kwargs_shear]

# ##########################################################################################################
# ##########################################################################################################

# kwargs_fit_s1, precision_s1 = solver4Point.constraint_lensmodel(x_pos=x_image_s1_F200LP, y_pos=y_image_s1_F200LP, 
#                                                           kwargs_list=kwargs_lens_list, xtol=1.49012e-12)

# print('kwargs_fit_s1:', kwargs_fit_s1)
# print('#'*15)
# print('#'*15)

# ##########################################################################################################
# ##########################################################################################################

# for x, y in zip(x_image_s1_F200LP, y_image_s1_F200LP):
#     x_image, y_image = x, y
#     print('initial image position:', x_image, y_image)

#     alpha_x, alpha_y = lensModel.alpha(x_image, y_image, kwargs_fit_s1)

#     x_source, y_source = x_image-alpha_x, y_image-alpha_y

#     print('source:', x_source, y_source)

#     x_image, y_image = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_fit_s1, sourcePos_x=x_source, sourcePos_y=y_source, 
#                                                                     min_distance=0.01, search_window=10, precision_limit=10**(-10), num_iter_max=100)
#     print('image positions: ', x_image, y_image)
#     print('#'*15)

### some plots

In [ ]:
# # make a pixel grid suited for plotting
# from lenstronomy.Util import util



# x_grid, y_grid = util.make_grid(numPix=500, deltapix=0.01)

# # plot convergence

# kappa_grid = lensModel.kappa(x_grid, y_grid, kwargs_fit_F140W)
# kappa2d_fit = util.array2image(kappa_grid)


# f, axes = plt.subplots(1, 1, figsize=(12, 4), sharex=False, sharey=False)


# ax = axes
# im = ax.matshow(np.log10(kappa2d_fit), origin='lower')
# ax.text(1, 1, 'convergence, F140W')
# ax.autoscale(False)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)

# plt.show()

# # plot magnification

# mag_grid = lensModel.magnification(x_grid, y_grid, kwargs_fit_F140W)
# mag2d_fit = util.array2image(mag_grid)


# import matplotlib.pyplot as plt
# %matplotlib inline
# from mpl_toolkits.axes_grid1 import AxesGrid, make_axes_locatable

# f, axes = plt.subplots(1, 1, figsize=(12, 4), sharex=False, sharey=False)


# ax = axes
# im = ax.matshow(mag2d_fit, origin='lower', vmin=-10, vmax=10)
# ax.text(1, 1, 're-fit model with solver and substructure, F140W')
# ax.autoscale(False)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)


# plt.show()




In [ ]:
# # make a pixel grid suited for plotting
# from lenstronomy.Util import util



# x_grid, y_grid = util.make_grid(numPix=500, deltapix=0.01)

# # plot convergence

# kappa_grid = lensModel.kappa(x_grid, y_grid, kwargs_fit_F200LP)
# kappa2d_fit = util.array2image(kappa_grid)


# f, axes = plt.subplots(1, 1, figsize=(12, 4), sharex=False, sharey=False)


# ax = axes
# im = ax.matshow(np.log10(kappa2d_fit), origin='lower')
# ax.text(1, 1, 'convergence, F200LP')
# ax.autoscale(False)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)

# plt.show()

# # plot magnification

# mag_grid = lensModel.magnification(x_grid, y_grid, kwargs_fit_F200LP)
# mag2d_fit = util.array2image(mag_grid)


# import matplotlib.pyplot as plt
# %matplotlib inline
# from mpl_toolkits.axes_grid1 import AxesGrid, make_axes_locatable

# f, axes = plt.subplots(1, 1, figsize=(12, 4), sharex=False, sharey=False)


# ax = axes
# im = ax.matshow(mag2d_fit, origin='lower', vmin=-10, vmax=10)
# ax.text(1, 1, 're-fit model with solver and substructure, F200LP')
# ax.autoscale(False)
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)
# plt.colorbar(im, cax=cax)


# plt.show()


